In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# Simulated time series data
time = np.arange(0, 100, 0.1)
sin_wave = np.sin(time) + np.sin(time * 0.05) + np.sin(time * 0.12) 
sin_wave_anomalous = sin_wave.copy()
sin_wave_anomalous[300:500] = 3  # Anomaly

# Prepare training data
train_data = sin_wave[:700]
test_data = sin_wave[700:]
test_data_anomalous = sin_wave_anomalous[700:]

train_data = train_data.reshape(-1, 1)
test_data = test_data.reshape(-1, 1)
test_data_anomalous = test_data_anomalous.reshape(-1, 1)

# Create Autoencoder model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(train_data.shape[1])
])

model.compile(optimizer='adam', loss='mse')
model.fit(train_data, train_data, epochs=50, batch_size=16, validation_split=0.1)

# Evaluate model and detect anomalies
reconstruction = model.predict(test_data)
reconstruction_anomalous = model.predict(test_data_anomalous)

mse_normal = np.mean(np.power(test_data - reconstruction, 2), axis=1)
mse_anomalous = np.mean(np.power(test_data_anomalous - reconstruction_anomalous, 2), axis=1)

# Plotting the error
plt.figure(figsize=(10, 6))
plt.hist([mse_normal, mse_anomalous], bins=50, alpha=0.5, label=['Normal', 'Anomalous'])
plt.legend(loc='upper right')
plt.title('Distribution of Reconstruction Error')
plt.show()
